# InfoKost Web Scraper
### Dewa Ayu Mutiara Kirana Praba Dewi (18220084)
### Tugas Seleksi Calon Asisten Lab Basis Data

# Import semua library yang digunakan

In [6]:
from bs4 import BeautifulSoup
from urllib.request import urlopen as uReq, Request
import time
import json
from geopy.geocoders import Nominatim
geocoder = Nominatim(user_agent = 'infokostscraper')


# Membuat Kelas Data Kost

In [7]:
class DataKost:
    def __init__(self, nama_kost, alamat, daerah, kecamatan, kota, latitude, longitude, tipe_kost, tipe_kamar, luas_kamar, fasilitas_kamar, isKamarMandiDalam, fasilitas_kamar_mandi, fasilitas_gedung, harga_per_bulan):
        self.nama_kost = nama_kost
        self.alamat = alamat
        self.daerah = daerah
        self.kecamatan = kecamatan
        self.kota = kota
        self.latitude = latitude
        self.longitude = longitude
        self.tipe_kost = tipe_kost
        self.tipe_kamar = tipe_kamar
        self.luas_kamar = luas_kamar
        self.fasilits_kamar = fasilitas_kamar
        self.fasilitas_kamar_mandi = fasilitas_kamar_mandi
        self.isKamarMandiDalam = isKamarMandiDalam
        self.fasilitas_gedung = fasilitas_gedung
        self.harga_per_bulan = harga_per_bulan
    
    def printObject(self):
        print(f'''
        Nama Kost              : {self.nama_kost}
        Alamat                 : {self.alamat}
        Daerah                 : {self.daerah}
        Kecamatan              : {self.kecamatan}
        Kota                   : {self.kota}
        Latitude               : {self.latitude}
        Longitude              : {self.longitude}
        Tipe Kost              : {self.tipe_kost}
        Tipe Kamar             : {self.tipe_kamar}
        Luas Kamar             : {self.luas_kamar}
        Fasilitas Kamar        : {self.fasilits_kamar}
        Fasilitas Kamar Mandi  : {self.fasilitas_kamar_mandi}
        Is Kamar Mandi Dalam   : {self.isKamarMandiDalam}
        Fasilitas Gedung       : {self.fasilitas_gedung}
        Harga Per Bulan        : {self.harga_per_bulan}
        ''')

# Fungsi untuk Data Manipulation

In [8]:
#fungsi untuk mengubah format harga dari string menjadi integer
#contoh : Rp3.000.000 --> 3000000
def harga_to_integer(str):
    num = int(str[3:-7].replace(".", ""))
    return num

#fungsi untuk mengubah format luas kamar dari string menjadi integer
def luas_to_float(str):
    if 'm' == str[12]:
        #format Luas kamar : m2
        #dianggap None dan nanti akan di pass karena menandakan tidak terdapat data luas kamar
        num = None
    elif 'dan' in str:
        #format suatu tipe yang memiliki 2 luas kamar
        #dianggap None dan nanti akan di pass karena ambigu dan rancu
        num = None
    elif '-' in str:
        #format suatu tipe yang memiliki luas kamar beragam dan berupa range
        #dianggap None dan nanti akan di pass karena ambigu dan rancu
        num = None
    else: 
        if '.' in str:
            if 'x' in str:
                #format Luas kamar : a.b x c.d
                #dianggap None dan nanti akan di pass (kasus khusus)
                num = None
            else:
                #format luas kamar berupa float
                num = float(str[12:-2])
        elif ',' in str:
            if 'x' in str:
                #format Luas kamar : a,b x c,d
                #dianggap None dan nanti akan di pass (kasus khusus)
                num = None
            else:
                #format luas kamar berupa float, menggunakan ','
                num = float(str[12:-2].replace(",", "."))
        else:
            if 'x' in str:
                if 'x' == str[13]:
                    #format perkalian akan dikalikan
                    num = (float(str[12]) * float(str[14]))
                else:
                    #kasus khusus akan di pass
                    num = None
            else:
                num = float(str[12:-2])
    return num

# Fungsi untuk scrape web infokost

In [9]:
def get_data_kost(listOfKost, firstpage, lastpage):
    page = firstpage
    while page <= lastpage :
        try :
            url = "https://infokost.id/search-results/page/"+str(page)+"/" #base url yang dipakai untuk setiap page
            header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64); Basis Data/Admin Basis Data/basisdata@std.stei.itb.ac.id'}
            req_page = Request(url, headers=header)
            uClient = uReq(req_page)
            html_text = uClient.read()
            uClient.close()
            soup_page = BeautifulSoup(html_text, "lxml")
            
            print(" ")
            print(f'[SCRAPING] {url} \n')
            kost_list = soup_page.find_all('div', class_='item-wrap infobox_trigger homey-matchHeight') 

            for kost in kost_list[:-5]:
                try:
                    specific_url = kost.h2.a["href"] #spesific url dari tiap item kost untuk mendapatkan info secara detail
                    spesific_page = Request(specific_url, headers=header)
                    uClient = uReq(spesific_page)
                    specific_html_text = uClient.read()
                    uClient.close()
                    subsoup = BeautifulSoup(specific_html_text, "lxml")
                    featured_list_kamar = subsoup.find('div',{'class': 'item-row item-list-view'})
                    list_kamar = featured_list_kamar.find_all('div', class_='item-wrap infobox_trigger homey-matchHeight')
                    
                    for kamar in list_kamar:
                        #array untuk menampung list fasilitas kamar dan gedung
                        arr_fasilitas_kamar = []
                        arr_fasilitas_gedung = []

                        #mengambil info luas kamar
                        featured_luas_kamar = kamar.find('address', class_='item-address').text 
                        luas_kamar = luas_to_float(featured_luas_kamar) #convert ke float
                        if luas_kamar is None:
                            #apabila luas kamar tidak ada atau berupa kasus khusus penyebab ambigu/rancu maka akan di pass
                            #tidak mengambil bad data
                            pass
                        else:
                            #mengambil info nama kost
                            nama_kost = subsoup.find('h1', class_='listing-title').text.strip()
                            if nama_kost == '':
                                #tidak mengambil bad data
                                pass
                            else:
                                #mengambil info alamat kost
                                alamat = subsoup.find('address', class_='item-address').text.strip()
                                if alamat == '':
                                    #tidak mengambil bad data
                                    pass
                                else:
                                    #mengambil info daerah kost
                                    daerah = kost.find('address', class_='item-address').text
                                    arr_daerah = daerah.split(",") #menyimpan info daerah secara spesifik ke dalam array
                                    if len(arr_daerah) == 2 : 
                                        #default daerah adalah kecamatan, kota (length = 2)
                                        #apabila length tidak = 2 maka ada info yang kurang ataupun tidak cocok dengan format, dan akan di pass
                                        kecamatan = arr_daerah[0].strip() #menyimpan info daerah secara spesifik, yaitu kecamatan
                                        kota = arr_daerah[1].strip() #menyimpan info daerah secara spesifik, yaitu kota
                                        if kecamatan == '':
                                            #kasus khusus, length = 2 tetapi kecamatannya kosong maka akan di pass (bad data)
                                            pass
                                        else:
                                            #mencari koordinat daerah
                                            coordinates = geocoder.geocode(str(daerah)) 
                                            if coordinates is not None:
                                                latitude = coordinates.latitude
                                                longitude = coordinates.longitude
                                            else:
                                                coordinates_alt = geocoder.geocode(str(kota)) 
                                                if coordinates_alt is not None:
                                                    latitude = coordinates_alt.latitude
                                                    longitude = coordinates_alt.longitude
                                                else:
                                                    latitude = -2.4833826 #latitude default Indonesia
                                                    longitude = 117.8902853 #longitude default Indonesia
                                            #mengambil info tipe kost, apakah kost campur, putra, atau putri
                                            tipe_kost = kost.find('div', class_='col-sm-5 col-xs-6 no-l-pad').text.strip()
                                            #mengambil info tipe kamar 
                                            tipe_kamar = kamar.find('h2', class_='title roomtitle').text.strip()
                                            #mengambil info fasilitas kamar yang disediakan
                                            list_fasilitas_kamar = kamar.find('ul', class_='detail-list-disc no-pad').find_all('li')
                                            for faskamar in list_fasilitas_kamar:
                                                arr_fasilitas_kamar.append(faskamar.text.strip())
                                            if arr_fasilitas_kamar is not []:
                                                fasilitas_kamar = ', '.join(arr_fasilitas_kamar) #menyimpan info fasilitas kamar yang ada
                                                #mengambil info fasilitas kamar mandi
                                                fasilitas_kamar_mandi = kamar.find('span', class_='item-label miximgiconlabel').text
                                                arr_fasilitas_km = fasilitas_kamar_mandi.split(",")
                                                arr_km_word = arr_fasilitas_km[0].split(" ")
                                                if len(arr_km_word) == 3:
                                                    #mengambil info apakah termasuk kamar mandi dalam
                                                    if arr_km_word[2] == ("dalam" or "dalem"):
                                                        isKamarMandiDalam = True
                                                    else:
                                                        isKamarMandiDalam = False
                                                    #mengambil info fasilitas gedung
                                                    list_fasilitas_gedung = subsoup.find('ul', class_='detail-list detail-list-2-cols').find_all('li')
                                                    for fasgedung in list_fasilitas_gedung:
                                                        arr_fasilitas_gedung.append(fasgedung.text.strip())
                                                    if arr_fasilitas_gedung is not []:
                                                        fasilitas_gedung = ', '.join(arr_fasilitas_gedung) #menyimpan info fasilitas gedung yang ada
                                                        #mengambil info harga per bulan
                                                        harga = kamar.find('span', class_='item-price').text
                                                        harga_per_bulan = harga_to_integer(harga)
                                                        newKost = DataKost(nama_kost, alamat, daerah, kecamatan, kota, latitude, longitude, tipe_kost, tipe_kamar, luas_kamar, fasilitas_kamar, isKamarMandiDalam, fasilitas_kamar_mandi, fasilitas_gedung, harga_per_bulan)
                                                        newKost.printObject()
                                                        listOfKost.append(newKost)
                                                    else:
                                                        #apabila fasilitas gedung tidak dicantumkan (bad data)
                                                        pass
                                                else:
                                                    #tidak terdapat info apakah termasuk kamar mandi dalam (bad data)
                                                    pass
                                            else:
                                                #apabila fasilitas kamar tidak dicantumkan (bad data)
                                                pass
                                    else:
                                        pass
                            time.sleep(3)
                except (AttributeError):
                    pass
                except Exception as e:
                    print(e)
            print(f"[Scraping halaman {page} berhasil!]")
            page +=1
        except Exception as e:
            print("\n Error: ", e)
            print("Error or Page finished")
            break

    print(" ")
    print(f"[Scraping dari halaman {firstpage} sampai halaman {lastpage} telah berhasil]")


# Scraping infokost.id part 1

In [169]:
#melakukan scraping page 1 - 50
list_data_kost1 = []
get_data_kost(list_data_kost1, 1, 50)

 
[SCRAPING] https://infokost.id/search-results/page/1/ 

[Scraping halaman 1 berhasil!]
 
[SCRAPING] https://infokost.id/search-results/page/2/ 

[Scraping halaman 2 berhasil!]
 
[SCRAPING] https://infokost.id/search-results/page/3/ 

[Scraping halaman 3 berhasil!]
 
[SCRAPING] https://infokost.id/search-results/page/4/ 

[Scraping halaman 4 berhasil!]
 
[SCRAPING] https://infokost.id/search-results/page/5/ 

[Scraping halaman 5 berhasil!]
 
[SCRAPING] https://infokost.id/search-results/page/6/ 

[Scraping halaman 6 berhasil!]
 
[SCRAPING] https://infokost.id/search-results/page/7/ 

[Scraping halaman 7 berhasil!]
 
[SCRAPING] https://infokost.id/search-results/page/8/ 

[Scraping halaman 8 berhasil!]
 
[SCRAPING] https://infokost.id/search-results/page/9/ 

[Scraping halaman 9 berhasil!]
 
[SCRAPING] https://infokost.id/search-results/page/10/ 

[Scraping halaman 10 berhasil!]
 
[SCRAPING] https://infokost.id/search-results/page/11/ 

[Scraping halaman 11 berhasil!]
 
[SCRAPING] http

In [190]:
len(list_data_kost1)
#mengecek datanya berhasil diambil dan tersimpan di list_data_kost1

1204

# Scraping infokost.id part 2

In [192]:
#melakukan scraping page 51 - 100
list_data_kost2 = []
get_data_kost(list_data_kost2, 51, 100)

 
[SCRAPING] https://infokost.id/search-results/page/51/ 

[Scraping halaman 51 berhasil!]
 
[SCRAPING] https://infokost.id/search-results/page/52/ 

[Scraping halaman 52 berhasil!]
 
[SCRAPING] https://infokost.id/search-results/page/53/ 

[Scraping halaman 53 berhasil!]
 
[SCRAPING] https://infokost.id/search-results/page/54/ 

[Scraping halaman 54 berhasil!]
 
[SCRAPING] https://infokost.id/search-results/page/55/ 

[Scraping halaman 55 berhasil!]
 
[SCRAPING] https://infokost.id/search-results/page/56/ 

[Scraping halaman 56 berhasil!]
 
[SCRAPING] https://infokost.id/search-results/page/57/ 

[Scraping halaman 57 berhasil!]
 
[SCRAPING] https://infokost.id/search-results/page/58/ 

[Scraping halaman 58 berhasil!]
 
[SCRAPING] https://infokost.id/search-results/page/59/ 

[Scraping halaman 59 berhasil!]
 
[SCRAPING] https://infokost.id/search-results/page/60/ 

[Scraping halaman 60 berhasil!]
 
[SCRAPING] https://infokost.id/search-results/page/61/ 

[Scraping halaman 61 berhasil!]

In [193]:
len(list_data_kost2)
#mengecek datanya berhasil diambil dan tersimpan di list_data_kost2

986

# Data Preprocessing

In [195]:
allKost = list_data_kost1 + list_data_kost2 #merge data keseluruhan
len(allKost) #cek length keseluruhan

2190

# Output ke format JSON

In [ ]:
#convert list kosan to json
with open('Data Kost.json', 'w') as data:
    json.dump([obj.__dict__ for obj in allKost], data, indent=4)